In [15]:
#tensor flow guide book https://archive.org/details/ai-ml/page/24/mode/2up
import tensorflow as tf # for ML and we will be using a tensorflow data set/ the mnist number data set
import numpy as np #working with arrays
import matplotlib.pyplot as plt # for visualization
import cv2 #for computer vision
import os

In [16]:
_data = tf.keras.datasets.mnist #this is a training dataset generated by MNIST for number recoginition first data set made by mnist
#due to this data being already split up in test adn training data we can call this below without splitting or data preprocessing
(x_train, y_train), (x_test, y_test)= _data.load_data() #x is the pixel data, y is the classification/number


In [29]:
#now lets normalize the data/grey scaling 
"""gray scale pixel has value of  255  normalizing would make it between 0 and 1 
it will be scaling things down the value of 125 might look like 0.5"""

"""axis=-1:
When axis=-1, normalization is performed along the last axis of the input data.
In the context of a 2D tensor (like a matrix), where the first axis corresponds to rows and the second axis corresponds to columns, axis=-1 implies normalization is performed along the columns.
This means that each feature (column) in the input data is independently normalized across the samples.
axis=1:
When axis=1, normalization is performed along the second axis of the input data.
In the context of a 2D tensor, where the first axis corresponds to rows and the second axis corresponds to columns, axis=1 implies normalization is performed along the rows.
This means that each sample (row) in the input data is independently normalized across its features.

Normalization Along Rows:
With axis=0, normalization is applied along each feature (column) across all the samples (rows).
Each feature's mean and standard deviation are calculated across all the samples, considering all the values of that feature across all samples.
The normalization is then applied to each sample's feature values based on these aggregated statistics.

"""
x_train= tf.keras.utils.normalize(x_train, axis= 1)
x_test = tf.keras.utils.normalize(x_test, axis =1)

In [23]:
#check the shape of what x_train and x_test is
print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

Training data shape: (60000, 28, 28)
Test data shape: (10000, 28, 28)


In [31]:
# Plot some of the images from the training set
"""
plt.figure(figsize=(10, 10))
for i in range(25):  # Plotting the first 25 images
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.axis('off')  # Turn off axis labels
    plt.title(f'Label: {y_train[i]}')  # Show the label as the title
plt.show()
"""

"\nplt.figure(figsize=(10, 10))\nfor i in range(25):  # Plotting the first 25 images\n    plt.subplot(5, 5, i + 1)\n    plt.imshow(x_train[i], cmap='gray')\n    plt.axis('off')  # Turn off axis labels\n    plt.title(f'Label: {y_train[i]}')  # Show the label as the title\nplt.show()\n"

In [37]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28,28))) #flattens the data turns the pixels from 28x28 to a line between 784
#flatten turns it from 2-D to a 1-D array
